In [4]:
names = open("names.txt").read().splitlines()
names[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [5]:
# Exercises:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
# E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
# E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
# E06: meta-exercise! Think of a fun/interesting exercise and complete it.

In [6]:
# will start with trigram model using probabality
# given two words i need to egt th ecount of 3rd one
# here i will be using counter instead of dictionary
from collections import Counter
counter = Counter()
for name in names:
    name_modified = "."+name+"."
    counter.update(["".join((char, char_next1, char_next2)) for char, char_next1, char_next2 in zip(name_modified, name_modified[1:], name_modified[2:])])

# i wanted to plot this in 3d graph, , it looks a lot of effor to show it meaningfully.
# for now i will try to work, without graph
# so after geting the count i think i need to put them into tensor, and then
# after broadcasting i can get the probablity

In [8]:
# import torch

# N = torch.zeros((27,27,27), dtype=torch.int32)
#how do i fill the values counts in the tensor
for item in counter.items():
    print(item)

('.em', 288)
('emm', 100)
('mma', 72)
('ma.', 174)
('.ol', 104)
('oli', 69)
('liv', 54)
('ivi', 78)
('via', 147)
('ia.', 903)
('.av', 243)
('ava', 161)
('va.', 93)
('.is', 124)
('isa', 142)
('sab', 76)
('abe', 173)
('bel', 201)
('ell', 822)
('lla', 337)
('la.', 684)
('.so', 152)
('sop', 21)
('oph', 37)
('phi', 61)
('hia', 81)
('.ch', 352)
('cha', 236)
('har', 329)
('arl', 287)
('rlo', 44)
('lot', 14)
('ott', 34)
('tte', 121)
('te.', 175)
('.mi', 393)
('mia', 95)
('.am', 384)
('ame', 226)
('mel', 188)
('eli', 537)
('lia', 518)
('.ha', 505)
('arp', 8)
('rpe', 5)
('per', 77)
('er.', 683)
('.ev', 154)
('eve', 142)
('vel', 76)
('ely', 353)
('lyn', 976)
('yn.', 953)
('.ab', 190)
('abi', 76)
('big', 15)
('iga', 35)
('gai', 18)
('ail', 259)
('il.', 119)
('emi', 160)
('mil', 259)
('ily', 203)
('ly.', 270)
('.el', 488)
('liz', 81)
('iza', 93)
('zab', 40)
('bet', 61)
('eth', 114)
('th.', 178)
('ila', 279)
('ave', 166)
('ver', 160)
('ery', 84)
('ry.', 201)
('sof', 18)
('ofi', 19)
('fia', 20)
('.ca

In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 16.3 MB/s  0:00:24m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 20.8 MB/s  0:00:16m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.5 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 36.8 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 34.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 18.9 MB/s  0:00:20m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 36.9 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 36.0 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 35.5 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 33.1 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━